# Fetch Data



### Set env variables

assumes that `SOCRATA_APP_TOKEN`, `SOCRATA_API_KEY_ID`, and `SOCRATA_API_KEY_SECRET` are set in `.env`

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

### Load Packages

In [ ]:
import os
import sys

INGESTION_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, INGESTION_PATH)

from ingestion import fetch

### Full Pull of Data
Takes about 48 minutes

In [3]:
# await fetch.fetch_all_data(save = True)

### Incremental Pull of Data

In [4]:
# await fetch.fetch_current_month(save = True)

### Old COde

In [ ]:
# def _create_socrata_client(api_endpoint, app_token, username, password):
#     """"
    
#     """
#     client = Socrata(api_endpoint, app_token=app_token, username=username, password=password, timeout = 600)
#     return client


In [ ]:
# def _fetch_and_save_month_sync(year: int, month: int):
#     """Runs in a worker thread. Creates its own Socrata client (no sharing)."""
#     # per-thread client
#     # client = Socrata("data.cityofnewyork.us", APP_TOKEN)
#     client = _create_socrata_client(api_endpoint="data.cityofnewyork.us", app_token=APP_TOKEN, username=API_KEY_ID, password=API_KEY_SECRET)

#     start = f"{year}-{month:02d}-01T00:00:00"
#     if month == 12:
#         end = f"{year+1}-01-01T00:00:00"
#     else:
#         end = f"{year}-{month+1:02d}-01T00:00:00"

#     where_clause = f"{date_column} >= '{start}' AND {date_column} < '{end}'"
#     print(f"Fetching {year}-{month:02d} ...")

#     # client.get_all handles paging internally (blocking)
#     results = list(client.get_all(dataset_id, where=where_clause))

#     if results:
#         df = pd.DataFrame.from_records(results)
#         # write to year/month partition
#         file_path = os.path.join(output_dir, f"year={year}/month={month:02d}/part-0000.parquet")
#         os.makedirs(os.path.dirname(file_path), exist_ok=True)
#         # choose your engine; fastparquet avoids some pyarrow quirks in notebooks
#         df.to_parquet(file_path, index=False, engine="fastparquet")
#         print(f"Saved {file_path} ({len(df):,} rows)")

#         # clean up memory
#         del df
#         del results
#         gc.collect()
#     else:
#         print(f"No data for {year}-{month:02d}")

# async def fetch_and_save(year: int, month: int, sem: asyncio.Semaphore):
#     async with sem:
#         # run the sync worker in a thread
#         await asyncio.to_thread(_fetch_and_save_month_sync, year, month)

# async def pull_latest_data():
#     sem = asyncio.Semaphore(MAX_CONCURRENCY)
#     tasks = [fetch_and_save(y, m, sem) for y in years for m in months]
#     await asyncio.gather(*tasks)


In [ ]:
# dataset_id = "erm2-nwe9"
# date_column = "created_date"
# output_dir = "../data/311-service-requests"
# os.makedirs(output_dir, exist_ok=True)

# years = range(2010, 2026)   # e.g., 2010–2011
# months = range(1, 13)       # 1..12
# MAX_CONCURRENCY = 20         # tune for your machine / API limits

In [ ]:
# await pull_latest_data()